In [1]:
import json
import pandas as pd
from tqdm import tqdm

In [2]:
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/acl25_gpt4/shuffled_with_module_head/graph.jsonl'
graph = pd.read_json(graph_path, lines=True)

In [4]:
graph.iloc[0]['nodes'], graph.iloc[0]['edge_attrs'], graph.iloc[0]['connectivity']

([{'id': 0, 'content': 'A', 'type': 'input port'},
  {'id': 1, 'content': 'B', 'type': 'input port'},
  {'id': 2, 'content': 'A_greater', 'type': 'output port'},
  {'id': 3, 'content': 'A_equal', 'type': 'output port'},
  {'id': 4, 'content': 'A_less', 'type': 'output port'},
  {'id': 5, 'content': 'higher_cmp', 'type': 'submodule'},
  {'id': 6, 'content': 'lower_cmp', 'type': 'submodule'}],
 [],
 [[1, 0, 1, 0, 6, 6, 6], [5, 5, 6, 6, 2, 3, 4]])

In [6]:
print(graph.iloc[0])

nodes           [{'id': 0, 'content': 'A', 'type': 'input port...
edge_attrs                                                     []
connectivity       [[1, 0, 1, 0, 6, 6, 6], [5, 5, 6, 6, 2, 3, 4]]
Name: 0, dtype: object


In [3]:
len(graph)

1326

In [4]:
print(graph.iloc[0]['nodes'])
print(graph.iloc[0]['connectivity'])


[{'id': 0, 'content': 'A', 'type': 'input port'}, {'id': 1, 'content': 'B', 'type': 'input port'}, {'id': 2, 'content': 'A_greater', 'type': 'output port'}, {'id': 3, 'content': 'A_equal', 'type': 'output port'}, {'id': 4, 'content': 'A_less', 'type': 'output port'}, {'id': 5, 'content': 'higher_cmp', 'type': 'submodule'}, {'id': 6, 'content': 'lower_cmp', 'type': 'submodule'}]
[[1, 0, 1, 0, 6, 6, 6], [5, 5, 6, 6, 2, 3, 4]]


In [5]:
input_ports = []
output_ports = []
submodules = []

In [6]:
for i in tqdm(range(len(graph))):
    nodes = graph.iloc[i]['nodes']
    for node in nodes:
        if node['type'] == 'input port':
            input_ports.append(node['content'])
        elif node['type'] == 'output port':
            output_ports.append(node['content'])
        elif node['type'] == 'submodule':
            submodules.append(node['content'])
            

100%|██████████| 1326/1326 [00:00<00:00, 29045.43it/s]


In [7]:
len(input_ports), len(output_ports), len(submodules)

(4461, 2406, 4519)

In [8]:
input_ports = list(set(input_ports))
output_ports = list(set(output_ports))
submodules = list(set(submodules))

In [9]:
len(input_ports), len(output_ports), len(submodules)

(420, 394, 1509)

In [10]:
import random
from collections import defaultdict

# Predefined names for each type
BASE_NAMES = {
    'input port': input_ports,
    'output port': output_ports,
    'submodule': submodules
}
# Global counters to track name usage across the dataset
name_usage = defaultdict(int)

In [ ]:
def get_balanced_name(name_type):
    """Select a name of the given type, prioritizing names that have been used the least so far."""
    names = BASE_NAMES[name_type]
    # Find the minimum usage count among the names
    min_usage = min(name_usage[name] for name in names)
    # Filter names that have the minimum usage count
    candidates = [name for name in names if name_usage[name] == min_usage]
    # Randomly select one of the candidates
    selected_name = random.choice(candidates)
    # Update the usage count
    name_usage[selected_name] += 1
    return selected_name

def generate_names(name_type, count):
    """Generate a list of names for a given type, ensuring balanced usage across the dataset."""
    return [get_balanced_name(name_type) for _ in range(count)]

def weighted_random_edge_count(min_edges, max_edges, limit=20):
    """Generate a random number of edges, favoring smaller values (especially below 25)."""
    # Assign higher weights to edge counts below 25
    weights = []
    for i in range(min_edges, max_edges + 1):
        if i < limit:
            weights.append(3.5)  # Higher weight for edge counts below 25
        else:
            weights.append(0.1)  # Lower weight for edge counts 25 and above
    # Normalize the weights
    total_weight = sum(weights)
    probabilities = [w / total_weight for w in weights]
    # Randomly select a number of edges based on the probabilities
    return random.choices(range(min_edges, max_edges + 1), probabilities)[0]

def construct_graph(input_ports, output_ports, submodules, max_edges=50):
    """Construct a graph with nodes and edges, ensuring constraints are satisfied and the number of edges tends to be small."""
    nodes = []
    node_id = 0

    # Add input ports
    for port in input_ports:
        nodes.append({'id': node_id, 'content': port, 'type': 'input port'})
        node_id += 1

    # Add output ports
    for port in output_ports:
        nodes.append({'id': node_id, 'content': port, 'type': 'output port'})
        node_id += 1

    # Add submodules
    for module in submodules:
        nodes.append({'id': node_id, 'content': module, 'type': 'submodule'})
        node_id += 1

    # Construct edges
    edges_set = set()  # Use a set to avoid duplicate edges
    source_nodes = []
    target_nodes = []

    # Helper function to add an edge
    def add_edge(src, tgt):
        if (src, tgt) not in edges_set:
            edges_set.add((src, tgt))
            source_nodes.append(src)
            target_nodes.append(tgt)

    # Ensure each input port points to at least one submodule
    for i in range(len(input_ports)):
        target_submodule = random.randint(len(input_ports) + len(output_ports), len(input_ports) + len(output_ports) + len(submodules) - 1)
        add_edge(i, target_submodule)

    # Ensure each submodule has at least one source node (from input port or another submodule)
    for j in range(len(submodules)):
        submodule_id = len(input_ports) + len(output_ports) + j
        # Randomly decide whether to add an edge from an input port or another submodule
        if len(input_ports) > 0 and (len(submodules) == 0 or random.choice([True, False])):
            # Add an edge from an input port
            src = random.randint(0, len(input_ports) - 1)
            add_edge(src, submodule_id)
        elif j > 0:
            # Add an edge from another submodule (with lower ID)
            src = len(input_ports) + len(output_ports) + random.randint(0, j - 1)
            add_edge(src, submodule_id)
        else:
            # If there are no input ports and this is the first submodule, add an edge from an output port
            src = random.randint(len(input_ports), len(input_ports) + len(output_ports) - 1)
            add_edge(src, submodule_id)

    # Ensure each output port has at least one source submodule
    for k in range(len(output_ports)):
        output_id = len(input_ports) + k
        src_submodule = random.randint(len(input_ports) + len(output_ports), len(input_ports) + len(output_ports) + len(submodules) - 1)
        add_edge(src_submodule, output_id)

    # Calculate the minimum number of edges required to satisfy constraints
    min_edges = len(edges_set)

    # Randomly choose the total number of edges, favoring smaller values (especially below 25)
    # print("min_edges", min_edges, "~", max_edges)
    

    total_edges = weighted_random_edge_count(min_edges, max_edges)
    # print("total_edges", total_edges)
    # Collect all possible edges that don't already exist
    remaining_edges = []
    # Input ports to submodules
    for i in range(len(input_ports)):
        for j in range(len(submodules)):
            if (i, len(input_ports) + len(output_ports) + j) not in edges_set:
                remaining_edges.append(('input_to_submodule', i, len(input_ports) + len(output_ports) + j))
    # Submodules to submodules
    for i in range(len(submodules)):
        for j in range(i + 1, len(submodules)):
            if (len(input_ports) + len(output_ports) + i, len(input_ports) + len(output_ports) + j) not in edges_set:
                remaining_edges.append(('submodule_to_submodule', len(input_ports) + len(output_ports) + i, len(input_ports) + len(output_ports) + j))
    # Submodules to output ports
    for i in range(len(submodules)):
        for j in range(len(output_ports)):
            if (len(input_ports) + len(output_ports) + i, len(input_ports) + j) not in edges_set:
                remaining_edges.append(('submodule_to_output', len(input_ports) + len(output_ports) + i, len(input_ports) + j))

    # Randomly select from the remaining edges until the total number of edges reaches total_edges
    while len(edges_set) < total_edges and remaining_edges:
        edge = random.choice(remaining_edges)
        edge_type, src, tgt = edge
        add_edge(src, tgt)
        remaining_edges.remove(edge)

    return nodes, [source_nodes, target_nodes]

def generate_dataset(num_graphs, max_edges=50):
    """Generate a dataset of graphs, ensuring balanced name usage and small edge count across the dataset."""
    dataset = []
    for _ in range(num_graphs):
        # num_inputs = random.randint(1, 18)
        num_inputs = weighted_random_edge_count(1, 18, limit=8)
        # num_outputs = random.randint(1, 9)
        num_outputs = weighted_random_edge_count(1, 9, limit=7)
        # num_submodules = random.randint(1, 16)
        num_submodules = weighted_random_edge_count(1, 16, limit=5)

        input_ports = generate_names('input port', num_inputs)
        output_ports = generate_names('output port', num_outputs)
        submodules = generate_names('submodule', num_submodules)

        nodes, edges = construct_graph(input_ports, output_ports, submodules, max_edges)
        dataset.append((nodes, edges))
    return dataset

# Example usage
dataset = generate_dataset(30, max_edges=48)
for i, (nodes, edges) in enumerate(dataset):
    print(f"Graph {i+1}:")
    print("Nodes:", nodes)
    print("Edges:", edges)
    print(f"Number of edges: {len(edges[0])}")
    print()

# Print name usage statistics
print("Name Usage Across Dataset:")
for name_type, names in BASE_NAMES.items():
    print(f"{name_type}:")
    for name in names:
        print(f"  {name}: {name_usage[name]} times")

min_edges 26 ~ 50
total_edges 26
min_edges 7 ~ 50
total_edges 16
min_edges 13 ~ 50
total_edges 14
min_edges 11 ~ 50
total_edges 13
min_edges 5 ~ 50
total_edges 7
min_edges 13 ~ 50
total_edges 13
min_edges 11 ~ 50
total_edges 15
min_edges 13 ~ 50
total_edges 14
min_edges 7 ~ 50
total_edges 10
min_edges 8 ~ 50
total_edges 12
min_edges 10 ~ 50
total_edges 15
min_edges 8 ~ 50
total_edges 19
min_edges 9 ~ 50
total_edges 14
min_edges 14 ~ 50
total_edges 17
min_edges 9 ~ 50
total_edges 9
min_edges 8 ~ 50
total_edges 18
min_edges 8 ~ 50
total_edges 10
min_edges 12 ~ 50
total_edges 15
min_edges 21 ~ 50
total_edges 36
min_edges 8 ~ 50
total_edges 14
min_edges 8 ~ 50
total_edges 18
min_edges 7 ~ 50
total_edges 13
min_edges 13 ~ 50
total_edges 18
min_edges 9 ~ 50
total_edges 13
min_edges 21 ~ 50
total_edges 44
min_edges 8 ~ 50
total_edges 9
min_edges 14 ~ 50
total_edges 14
min_edges 11 ~ 50
total_edges 19
min_edges 8 ~ 50
total_edges 19
min_edges 11 ~ 50
total_edges 12
Graph 1:
Nodes: [{'id': 0, '

In [12]:
graph.iloc[0]['connectivity']

[[1, 0, 1, 0, 6, 6, 6], [5, 5, 6, 6, 2, 3, 4]]

In [14]:
for i in tqdm(range(len(graph))):
    connectivity = graph.iloc[i]['connectivity']
    print("len(connectivity):", len(connectivity[0]), "====", i)

100%|██████████| 1326/1326 [00:00<00:00, 15230.88it/s]

len(connectivity): 7 ==== 0
len(connectivity): 24 ==== 1
len(connectivity): 12 ==== 2
len(connectivity): 12 ==== 3
len(connectivity): 8 ==== 4
len(connectivity): 16 ==== 5
len(connectivity): 7 ==== 6
len(connectivity): 4 ==== 7
len(connectivity): 40 ==== 8
len(connectivity): 6 ==== 9
len(connectivity): 17 ==== 10
len(connectivity): 13 ==== 11
len(connectivity): 20 ==== 12
len(connectivity): 9 ==== 13
len(connectivity): 17 ==== 14
len(connectivity): 20 ==== 15
len(connectivity): 17 ==== 16
len(connectivity): 10 ==== 17
len(connectivity): 12 ==== 18
len(connectivity): 11 ==== 19
len(connectivity): 3 ==== 20
len(connectivity): 17 ==== 21
len(connectivity): 4 ==== 22
len(connectivity): 2 ==== 23
len(connectivity): 34 ==== 24
len(connectivity): 18 ==== 25
len(connectivity): 7 ==== 26
len(connectivity): 9 ==== 27
len(connectivity): 1 ==== 28
len(connectivity): 10 ==== 29
len(connectivity): 10 ==== 30
len(connectivity): 13 ==== 31
len(connectivity): 10 ==== 32
len(connectivity): 20 ==== 33
le

In [18]:
len(graph.iloc[1213]['connectivity'][0])

34

In [20]:
input_max = 0
input_min = 100
output_max = 0
output_min = 100
submodule_max = 0
submodule_min = 100
for i in tqdm(range(len(graph))):
    nodes = graph.iloc[i]['nodes']
    input_num = 0
    output_num = 0
    submodule_num = 0
    for node in nodes:
        if node['type'] == 'input port':
            input_num += 1
        elif node['type'] == 'output port':
            output_num += 1
        elif node['type'] == 'submodule':
            submodule_num += 1
    if input_num == 0:
        print("input_num == 0", i)
    if output_num == 0:
        print("output_num == 0", i)
    if submodule_num == 0:
        print("submodule_num == 0", i)
    if input_num > input_max:
        input_max = input_num
    if input_num < input_min:
        input_min = input_num
    if output_num > output_max:
        output_max = output_num
    if output_num < output_min:
        output_min = output_num
    if submodule_num > submodule_max:
        submodule_max = submodule_num
    if submodule_num < submodule_min:
        submodule_min = submodule_num

print("input_max:", input_max)
print("input_min:", input_min)
print("output_max:", output_max)
print("output_min:", output_min)
print("submodule_max:", submodule_max)
print("submodule_min:", submodule_min)

100%|██████████| 1326/1326 [00:00<00:00, 21647.22it/s]

submodule_num == 0 7
submodule_num == 0 20
submodule_num == 0 23
submodule_num == 0 28
submodule_num == 0 35
submodule_num == 0 47
submodule_num == 0 109
submodule_num == 0 110
submodule_num == 0 127
submodule_num == 0 134
submodule_num == 0 142
submodule_num == 0 185
submodule_num == 0 199
submodule_num == 0 224
submodule_num == 0 240
submodule_num == 0 249
submodule_num == 0 251
submodule_num == 0 258
submodule_num == 0 262
submodule_num == 0 290
submodule_num == 0 317
submodule_num == 0 344
submodule_num == 0 357
input_num == 0 372
output_num == 0 372
submodule_num == 0 378
submodule_num == 0 410
submodule_num == 0 449
submodule_num == 0 469
submodule_num == 0 520
submodule_num == 0 522
submodule_num == 0 529
submodule_num == 0 534
submodule_num == 0 557
submodule_num == 0 566
submodule_num == 0 577
submodule_num == 0 581
submodule_num == 0 602
submodule_num == 0 647
submodule_num == 0 678
submodule_num == 0 719
submodule_num == 0 745
submodule_num == 0 759
submodule_num == 0 781
su